# Multi-swarm Environmen

We let swarms swap positions on the same map

Strategies: Random Priority, Priority-Based Search

## Setup Environment and instances

In [ ]:
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

In [ ]:
configs = []

# Solver config
solver_config = {
    "solver_name": "TEGSolver"
}

# Swarm 1
swarm_1_config = {}
swarm_1_config["capacity_percentage"] = 0.3
swarm_1_config["agent_radius"] = 1 

# Map config
swarm_1_config["map_type"] = "swarm" # Available maps: empty, corridor, obstacle, cross, swarm
swarm_1_config["num_samples"] = 100 # Available num_samples: 100, 200, 300, 400, 500

# config starts and goals

swarm_1_config["start_regions"] = [
    Polygon([[0, 40], [0, 0], [50, 0], [50, 40]]),
    # Polygon([[0, 160], [0, 120], [50, 120], [50, 160]])
]
# 
swarm_1_config["goal_regions"] = [
    Polygon([[160, 160], [160, 110], [200, 110], [200, 160]]),
]

swarm_1_config["num_starts"] = 3
swarm_1_config["num_goals"] = 3

configs.append(swarm_1_config)

# Swarm 2
swarm_2_config = {}
swarm_2_config["capacity_percentage"] = 0.3
swarm_2_config["agent_radius"] = 1 

# Map config
swarm_2_config["map_type"] = "swarm" # Available maps: empty, corridor, obstacle, cross, swarm
swarm_2_config["num_samples"] = 100 # Available num_samples: 100, 200, 300, 400, 500

# config starts and goals

swarm_2_config["start_regions"] = [
    Polygon([[0, 160], [0, 120], [50, 120], [50, 160]])
]
# 
swarm_2_config["goal_regions"] = [
    Polygon([[160, 40], [160, 0], [200, 0], [200, 40]]),
]

swarm_2_config["num_starts"] = 3
swarm_2_config["num_goals"] = 3

configs.append(swarm_2_config)

# Swarm 3
swarm_3_config = {}
swarm_3_config["capacity_percentage"] = 0.3
swarm_3_config["agent_radius"] = 1

# Map config
swarm_3_config["map_type"] = "swarm" # Available maps: empty, corridor, obstacle, cross, swarm
swarm_3_config["num_samples"] = 100 # Available num_samples: 100, 200, 300, 400, 500

# config starts and goals

swarm_3_config["start_regions"] = [
    Polygon([[160, 120], [160, 80], [200, 80], [200, 120]]),
]
# 
swarm_3_config["goal_regions"] = [
    Polygon([[0, 120], [0, 80], [50, 80], [50, 80]])
]

swarm_3_config["num_starts"] = 3
swarm_3_config["num_goals"] = 3

configs.append(swarm_3_config)

In [ ]:
from st_gaussian_prm.utils import load_instance
instances = [load_instance(instance_config, solver_config) for instance_config in configs]

In [ ]:
# Visualize instance
gaussian_prm = instances[0].gaussian_prm

fig, ax = gaussian_prm.visualize_map()

for color, instance in zip(['r', 'g', 'b', 'y'], instances):
    for start in instance.starts_idx:
        gaussian_prm.gaussian_nodes[start].visualize(ax, edgecolor=color)

    for goal in instance.goals_idx :
        gaussian_prm.gaussian_nodes[goal].visualize(ax, edgecolor=color, linestyle="--")
plt.show() 

## Planner

In [ ]:
from st_gaussian_prm.solvers.multiswarm import RandomPriority
from st_gaussian_prm.solvers.multiswarm import PriorityBasedSearch

solver = PriorityBasedSearch
# solver = RandomPriority

solution = solver(instances).solve()
if solution["success"]:
    paths = solution["paths"]
else:
    print("Failed to find solution.")

## APF

In [ ]:
from st_gaussian_prm.solvers.micro.apf import APF

apf_config = {
    "k_attr": 0.02,
    "k_rep": 100,
    "step_size": 0.5,
    "repulsion_radius": 2 + 0.5,
    "goal_chisq_threshold": 5.991, # 95% CI
    "max_rep_force": 10.0,
    "min_dist": 0.5,
    "max_step": 0.7,
    "damping": 0.7
}

trajectories = APF(solution["paths"], instances[0].gaussian_prm.gaussian_nodes, **apf_config).solve()

## Visualization

In [ ]:
# Static path
from matplotlib import pyplot as plt
fig, ax = gaussian_prm.visualize_map()

cmap = plt.get_cmap("rainbow")
# colors = [cmap(i / num_agents) for i in range(num_agj js)]

for i, path in enumerate(trajectories):
    x_coords = [loc[0] for loc in path]
    y_coords = [loc[1] for loc in path]
    ax.plot(x_coords, y_coords, '-', label='Path', color="blue", linewidth=0.8, alpha=0.5)

# for start in starts_idx:
    # gaussian_prm.gaussian_nodes[start].visualize(ax)
# 
# for goal in goals_idx:
    # gaussian_prm.gaussian_nodes[goal].visualize(ax, edgecolor="b")
# 
plt.show()

In [ ]:
# Animate path
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Circle

from IPython.display import Video

speed = 40

def animate_solution(agent_radius, paths, fig, ax, fig_path="."):
    """
        Visualize solution trajectory provided instance
    """
    agents = []
    cmap = plt.get_cmap("tab10")
    
    for i in range(len(paths)):
        loc = paths[i][0]
        circle = Circle((loc[0], loc[1]), radius=agent_radius, color=cmap(i % 10))
        agents.append(circle)
        ax.add_patch(circle)

    def init():
        return agents

    def update(frame):
        frame = frame * speed
        for agent, traj in zip(agents, paths):
            agent.set_center(traj[frame])
        return agents

    anim = FuncAnimation(fig, update, frames=len(paths[0]) // speed, 
                         init_func=init, blit=True, interval=100)
    anim.save(f"{fig_path}/apf_solution.mp4", writer='ffmpeg', fps=24)
    plt.close()

fig_path = "solutions"
fig, ax = gaussian_prm.visualize_map()
agent_radius=1
animate_solution(agent_radius, trajectories, fig, ax, fig_path=fig_path)
Video(filename="solutions/apf_solution.mp4", embed=True)